<a href="https://colab.research.google.com/github/PabloGuev17/DatosGeoEspaciales/blob/main/MCD_CDGE_tutorial1-IdentificarSistemasdeCoordenadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

# Tutorial 1: Herramientas computacionales para el manejo de datos vectoriales.

Este tutorial tiene como objetivos:
- Introducir las librerías clave de Python para el trabajo con datos vectoriales.
- Identificar los sistemas de referencia de coordenadas (SRC) en los cuales se presenta un dataset vectorial.
- Realizar transformaciones entre distintos SRCs.
- Crear geometrías y geodataframes.
- Superponer en un mismo gráfico distintas capas vectoriales.

## 1. Librerías de Python para operación y análisis de datos vectoriales.

En este primer tutorial comenzaremos a familiarizarnos con un conjunto de librerías fundamentales para trabajar con datos vectoriales en ciencia de datos geoespacial. Estas herramientas nos permitirán cargar, explorar, manipular y visualizar capas en formatos como Shapefile, GeoJSON o GeoPackage, estableciendo las bases para análisis más avanzados en los siguientes módulos.

- **Geopandas**: Extiende las capacidades de `pandas` para manejar datos espaciales, permitiendo cargar, manipular y analizar geometrías (puntos, líneas y polígonos) de forma intuitiva. Revisa acá la [documentación](https://geopandas.org/)

- **Shapely**: Librería para crear y operar con geometrías de manera eficiente. Permite realizar operaciones como intersecciones, uniones, buffers y mediciones espaciales. Revisa acá la [documentación](https://shapely.readthedocs.io/)

- **Contextily**: Facilita la incorporación de mapas base (basemaps) desde proveedores como OpenStreetMap, añadiendo contexto visual a los datos espaciales. Trabaja principalmente en proyección Web Mercator. Revisa acá la [documentación](https://contextily.readthedocs.io/ )

Pra todo el desarrollo del curso, es fundamental que te familiarizes con la documentación de cada librería, de maneras que puedas consultarla y encontrar rápidamente información y ejemplos útiles cada vez que necesites recordar el detalle de alguna función.

A continuación, importaremos estas librerías:

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import contextily as ctx


## 2. Datos: Cartografía pre-censal para la comuna de Viña del Mar, Chile.

Para este  primer ejercicio de análisis de datos vectoriales, trabajaremos un con tipo de datos esenciales para muchos problemas de ciencia de datos geoespacial: los datos censales.

Los datos y la cartografía censal son un pilar fundamental en la ciencia de datos geoespacial, ya que proporcionan información estandarizada, exhaustiva y territorialmente detallada sobre la población, las viviendas y las características socioeconómicas de un país. Estas capas permiten analizar patrones espaciales, identificar desigualdades territoriales, construir indicadores a distintas escalas (manzana, zona, comuna) y vincular variables demográficas con datos ambientales, de infraestructura o movilidad. En síntesis, la cartografía censal ofrece el marco espacial imprescindible para contextualizar y enriquecer cualquier análisis geoespacial orientado a la toma de decisiones informadas.

En cada país, existen instituciones encargadas de planificar, aplicar, analizar y publicar los datos derivados de los Censos de Población. En Chile, esta labor corresponde al Instituto Nacional de Estadísticas [(INE)](www.ine.cl), que recientemente publicó los datos del [Censo 2024](https://censo2024.ine.gob.cl/resultados/).

### 2.1 Cartografía Pre-censal.

En este primer tutorial, nos enfocaremos en la **cartografía precensal**: el conjunto de mapas, archivos espaciales y bases de datos que se elaboran antes de un censo con el fin de planificar y organizar el trabajo de levantamiento en terreno. Incluye la delimitación y actualización de unidades geográficas como manzanas, sectores censales, zonas y áreas operativas, además de la identificación de viviendas, edificaciones y límites administrativos.

Su objetivo es garantizar que el censo pueda realizarse de manera ordenada, completa y sin omisiones, proporcionando una base espacial precisa que permita asignar cargas de trabajo a los censistas, optimizar rutas y asegurar la cobertura total del territorio. Es, en esencia, la infraestructura geográfica que hace posible un conteo censal eficiente y de alta calidad.

La cartografía final del Censo 2024, estará disponible desde diciembre 2024 en la página web. Por ahora, utilizaremos una versión previamente descargada de la cartografía precensal, para la región de Valparaíso, Chile. En la carpeta [APC_2023_R5] encontrarás varias capas vectoriales correspondientes a:
- Límites comunales: `Comunal.shp`
- Ejes viales: `Eje_Vial.shp`
- Manzanas censales: `Manzana_Urbana.shp`

### 2.2 Datos de Establecimientos de Salud.

A la cartografía censal, agregaremos también la información de los centros de atención de salud, que es reportada semestralmente por el Departamento de Estadísticas e Información de Salud ([DEIS](https://deis.minsal.cl/)) del Ministerio de Salud de Chile.  En particular, usaremos el [Listado de Establecimientos 2025](https://repositoriodeis.minsal.cl/DatosAbiertos/Establecimientos%20DEIS%20MINSAL%2025-11-2025.xlsx).

**Todos los datos utilizados en este ejercicio están contenidos en la carpeta `data_T1`entregada junto a este Notebook.**


## 3. Ejercicios: Conociendo los datos censales.

### 3.1 Lectura e inspección de datos vectoriales.

In [ ]:
path='data_T1' #modificar a su ruta local

In [ ]:
comuna=gpd.read_file('%s/APC2023_R05/Comunal.shp'%path)
vial=gpd.read_file('%s/APC2023_R05/Eje_Vial.shp'%path)
manz=gpd.read_file('%s/APC2023_R05/Manzana_Urbana.shp'%path)

In [ ]:
comuna.crs

In [ ]:
vial.head()


Para identificar el SRC de un dataset cualquiera, inspeccionamos el atributo `.crs`

In [ ]:
manz.crs

El método `.info()` permite identificar las propiedades generales del GeoDataFrame:

In [ ]:
manz.info()

Y para graficar el dataset, usamos el método `plot()`de geopandas:

In [ ]:
comuna.plot()

In [ ]:
vina=comuna[comuna['N_COMUNA']=='VIÑA DEL MAR']
vina.plot();

In [ ]:
vina=comuna[comuna['N_COMUNA']=='VIÑA DEL MAR']
vial=vial[vial['N_COMUNA']=='VIÑA DEL MAR']
manz_vina=manz[manz['N_COMUNA']=='VIÑA DEL MAR']

### 3.2 Creación de geometrías a partir de datos tabulares.

Los datos de Establecimientos de Salud están en formato Excel, y contienen columnas con las coordenadas geográficas, que están dadas en el SRC EPSG:4326.

In [ ]:
salud=pd.read_excel('%s/Salud/EstablecimientosDEIS_MINSAL_22-10-2025.xlsx'%path)
salud=salud[salud['Nombre Comuna']=='Viña del Mar']

In [ ]:
salud.head()

In [ ]:
salud['lat']=salud['LATITUD [Grados decimales]']
salud['lon']=salud['LONGITUD [Grados decimales]']

In [ ]:
salud[['lon','lat']]

In [ ]:
salud_gdf=gpd.GeoDataFrame(salud,geometry=gpd.points_from_xy(salud['lon'],salud['lat'],crs=4326))

In [ ]:
salud_gdf.plot()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

crs='3857'

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
vial.to_crs(crs).plot(ax=ax,color='k',lw=0.5,label='Ejes viales')

manz_vina.to_crs(crs).boundary.plot(ax=ax,lw=0.2)


salud_gdf.to_crs(crs).plot(ax=ax,color='r',markersize=12,marker='s',label='Salud')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

ax.legend();
#fig.savefig('vinadelmar.jpg',dpi=300,bbox_inches='tight')

### 3.3 Creación de una geometría puntual.

In [ ]:
from shapely import Point

lat=-33.02310120139368
lon=-71.56716829999999

point=Point(lon,lat)

reloj=gpd.GeoDataFrame(geometry=[Point(lon,lat)],crs=4326)

In [ ]:
reloj.plot()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

crs='3857'

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
vial.to_crs(crs).plot(ax=ax,color='k',lw=0.5,label='Ejes viales')

manz_vina.to_crs(crs).boundary.plot(ax=ax,lw=0.2)


salud_gdf.to_crs(crs).plot(ax=ax,color='r',markersize=12,marker='s',label='Salud')
reloj.to_crs(crs).plot(ax=ax,marker='*',markersize=200,color='magenta',label='Reloj de Flores')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

ax.legend();

### 3.4 Exportar archivos.

El comando [`gpd.to_file()`](https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoDataFrame.to_file.html) permite exportar GeoDataFrames en distintos formatos vectoriales, como por ejemplo `geopackage`:

In [ ]:
vina.to_file('vm_comuna.gpkg',driver='GPKG')
manz_vina.to_file('vm_manz.gpkg',driver='GPKG')
salud_gdf.to_file('vm_salud.gpkg',driver='GPKG')
vial.to_file('vm_vial.gpkg',driver='GPKG')
